### Things to do in this module
1. Define the wall properly so that it is not even considered
--> changed the weight of wall to -100 and it works
2. Use epsilon in this instance

In [1]:
import numpy as np
import random

from sklearn.externals import joblib

In the gridworld we have only 4 possible actions

In [2]:
actions = ['up','down','left','right']
actionNum = [index for index, _ in enumerate(actions)]

# actionNum encodes: up, down, left and right as 0, 1, 2 and 3 respectively.
actionNum

[0, 1, 2, 3]

Let us consider the gridworld. And let it be a 4 by 4 pixaleted world

In [3]:
# our states will be 16 set of tuples 
states = []
for i in range(4):
    for j in range(4):
        states.append((i,j))
        
states

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)]

In [4]:
# 
statesAvailable = states[:]

# wallState = states[np.random.randint(0,len(statesAvailable))]
# posState = states[np.random.randint(0,len(statesAvailable))]
# negState = states[np.random.randint(0,len(statesAvailable))]

wallState = (2,1)
posState = (3,3)
negState = (2,3)

statesAvailable.pop(statesAvailable.index(wallState))
statesAvailable.pop(statesAvailable.index(posState))
statesAvailable.pop(statesAvailable.index(negState))

print(wallState, posState, negState, len(statesAvailable))

(2, 1) (3, 3) (2, 3) 13


In [5]:
# now that we have wall, pos and negative states defined
# we are going to create and update the reward matrix
r_matrix = np.zeros((4,4), dtype=np.int)
r_matrix[wallState] = -50
r_matrix[posState] = 100
r_matrix[negState] = -100

r_matrix

array([[   0,    0,    0,    0],
       [   0,    0,    0,    0],
       [   0,  -50,    0, -100],
       [   0,    0,    0,  100]])

In [6]:
# given coordinate and grid size it will tell if [up,down,left,right]
# is available
def actionsAvailable(state, row_max, col_max):
    row, col = state
    directions = [-1,-1,-1,-1]
    if row < row_max: directions[1] = 0
    if row > 0: directions[0] = 0
    if col < col_max: directions[3] = 0
    if col > 0: directions[2] = 0
    return directions

actionsAvailable((3,3),3,3)

[0, -1, 0, -1]

Here is the equation that will be used:

Q(S_t,A_t) = Q(S_t,A_t) + alpha X [R_t+1 + gamma X max{Q(S_t+1,A_t+1)}-Q(S_t,A_t)]

- some things to be congnicent off:
1. R_t+1 is the reward for the state that you are going to


In [7]:
row_max = 3
col_max = 3
epsilon = 0.2
alpha = 0.2
gamma = 0.9

# we are going to update q_matrix by putting -1 where we can't go
def initializeQMatrix(states, row_max, col_max):
    q_matrix = np.zeros((len(states),len(actionNum)), dtype=np.int)
    for state in states:
        qRowNum = int((((row_max +1)*(col_max + 1))/(row_max + 1))*state[0] + state[1])
        q_matrix[qRowNum] = actionsAvailable(state,3,3)
    return q_matrix

# this is not being used currently
def get_q_max(q_list):
    if -1 in q_list:
        q_temp = q_list[:]
        q_temp.pop(q_temp.index(-1))
        q_max = np.max(q_temp)
        return q_max
    else:
        q_max = np.max(q_list)

def possible_actions(q_list):
    q_actions = []
    for i,j in enumerate(q):
        if j != -1:
            q_actions.append(i)

    return q_actions
            
def choose_action(state, row_max, col_max):
    '''
    1. given state get its row number [qRowNum]
    2. for qRowNum get all column items (remember columnms signify up,down,left, right in that order)
    3. find max value amongnst all column(action) elements [qMax]
    4. get indices in from q where column elements == qMax [indices]
    5. random choice on any element in 'indices'
    '''
    qRowNum = int((16/(row_max + 1))*state[0] + state[1])
    q = list(q_matrix[qRowNum])
    # --initially put this function to ignore the wa
    # -- later adjusted the weight
    #qMax = get_q_max(q)
    qMax = np.max(q)
    
    indices = np.argwhere(q == qMax)
    chooseActionIndex = np.random.choice(indices.flatten('F'))
    return chooseActionIndex

# def choose_action(state, row_max, col_max):
#     '''
#     1. given state get its row number [qRowNum]
#     2. for qRowNum get all column items (remember columnms signify up,down,left, right in that order)
#     3. find max value amongnst all column(action) elements [qMax]
#     4. get indices in from q where column elements == qMax [indices]
#     5. random choice on any element in 'indices'
#     '''
#     qRowNum = int((16/(row_max + 1))*state[0] + state[1])
#     q = list(q_matrix[qRowNum])
    
#     # this section is choosing a random action 
#     if random.random() < epsilon:
#         possibleActions = possible_actions(q)
#         chooseActionIndex = np.random.choice(possibleActions)
#     else:
        
#         # --initially put this function to ignore the wa
#         # -- later adjusted the weight
#         #qMax = get_q_max(q)
#         qMax = np.max(q)

#         indices = np.argwhere(q == qMax)
#         chooseActionIndex = np.random.choice(indices.flatten('F'))
    
#     return chooseActionIndex

# now that we know the action to take (from choose_action)
# get state where action leads us
def stateTo(state, actionTaken):
    row, col = state
    if actionTaken == 0: row -= 1
    if actionTaken == 1: row += 1
    if actionTaken == 2: col -= 1
    if actionTaken == 3: col += 1
        
    return row,col

# Now get reward in this state
def getReward(state):
    return r_matrix[state[0], state[1]]

def updateQMatrix(state, actionChosen, stateSentTo, row_max, reward, alpha, gamma):
    
    # getting previous q value
    qRowNum = int((16/(row_max + 1))*state[0] + state[1])
    old_q = q_matrix[qRowNum, actionChosen]
    
    # getting best action from new q value
    qRowNum_latest_state = int((16/(row_max + 1))*stateSentTo[0] + stateSentTo[1])
    q = list(q_matrix[qRowNum_latest_state])
    qMax = np.max(q)
    #qMax = get_q_max(q)
    
    indices = np.argwhere(q == qMax)
    chooseActionIndex = np.random.choice(indices.flatten('F'))
    
    latest_q = q_matrix[qRowNum_latest_state, chooseActionIndex]
    q_matrix[qRowNum,actionChosen] = old_q + alpha*(reward + (gamma * latest_q) - old_q)
    
    return q_matrix

In [8]:
q_matrix = initializeQMatrix(states, row_max, col_max)

def process_qMatrix(state):
   
    while (state != posState) and (state != negState):
        actionChosen = choose_action(state, row_max, col_max)
        stateSentTo = stateTo(state, actionChosen)
        rewardInNewState = getReward(stateSentTo)
        #updateQMatrix(state, actionChosen, stateSentTo, row_max, reward, alpha, gamma):
        updateQMatrix(state, actionChosen, stateSentTo, row_max, rewardInNewState, alpha, gamma)

        print('start state: ', state)
        print('action taken: ', actionChosen)
        print('Action takes us to this state: ', stateSentTo)
        print('Reward at state we are in: ', rewardInNewState)
        print('Q matrix: ', q_matrix)
        
        state = stateSentTo

In [9]:
for _ in range(400):
    stateIndex = np.random.choice(len(statesAvailable))
    process_qMatrix(statesAvailable[stateIndex])

start state:  (1, 0)
action taken:  1
Action takes us to this state:  (2, 0)
Reward at state we are in:  0
Q matrix:  [[-1  0 -1  0]
 [-1  0  0  0]
 [-1  0  0  0]
 [-1  0  0 -1]
 [ 0  0 -1  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0 -1]
 [ 0  0 -1  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0 -1]
 [ 0 -1 -1  0]
 [ 0 -1  0  0]
 [ 0 -1  0  0]
 [ 0 -1  0 -1]]
start state:  (2, 0)
action taken:  3
Action takes us to this state:  (2, 1)
Reward at state we are in:  -50
Q matrix:  [[ -1   0  -1   0]
 [ -1   0   0   0]
 [ -1   0   0   0]
 [ -1   0   0  -1]
 [  0   0  -1   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0  -1]
 [  0   0  -1 -10]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0  -1]
 [  0  -1  -1   0]
 [  0  -1   0   0]
 [  0  -1   0   0]
 [  0  -1   0  -1]]
start state:  (2, 1)
action taken:  1
Action takes us to this state:  (3, 1)
Reward at state we are in:  0
Q matrix:  [[ -1   0  -1   0]
 [ -1   0   0   0]
 [ -1   0   0   0]
 [ -1   0   0  -1]
 [  0   0  -1   0]
 [ 

In [10]:
q_matrix

array([[ -1,   0,  -1,  39],
       [ -1,   0,   0,  48],
       [ -1,  58,   0,   0],
       [ -1,   0,  48,  -1],
       [  0,   0,  -1,  48],
       [  0, -10,   0,  58],
       [  0,  69,   0,   0],
       [  0, -20,  58,  -1],
       [  0,  58,  -1, -10],
       [  0,   0,   0,   0],
       [  0,  82,   0,   0],
       [  0,   0,   0,  -1],
       [  0,  -1,  -1,  69],
       [-10,  -1,   0,  82],
       [  0,  -1,   0,  96],
       [  0,  -1,   0,  -1]])

### test -----------------------------

In [12]:
# This section just gets rid of any case where we have -1 in the state
state = (2,0)
qRowNum = int( (16/4)*state[0] + state[1]   ) # row num for state = 8
q_matrix[qRowNum] # this is: array([ 0,  0, -1,  0])
q = list(q_matrix[qRowNum])
q # [0,0,-1,0]

[0, 0, -1, 0]

In [15]:
q_actions = []
for i,j in enumerate(q):
    if j != -1:
        q_actions.append(i)
        
q_actions

[0, 1, 3]

In [60]:
# we want to remove any actions that lead to -1
# if -1 in q:
#     q.pop(q.index(-1))
q # output here is [0,0,0]

# Now that we got rid of -1 we should be able to do the rest of the things with qMax
qMax = np.max(q)
if qMax == -1
qMax # this will be 0 since q was [0,0,0]

# get indices
indices = np.argwhere(q == qMax)
#indices # output is below


indices.flatten('F') # output: array([0, 1, 2], dtype=int32)
chooseActionIndex = np.random.choice(indices.flatten('F'))
#chooseActionIndex

0

In [82]:
def test_q_max(q_list):
    if -1 in q_list:
        q_temp = q_list[:]
        q_temp.pop(q_temp.index(-1))
        q_max = np.max(q_temp)
        return q_max
    
test1 = [-4,-5,-10,-1]
test2 = [1,2,3,-1]

test_q_max(test1)

-4

In [14]:
test1 = [-4,-5,-10,-1]
test2 = [1,2,3,-1]

def get_q_max(q_list):
    if -1 in q_list:
        q_temp = q_list[:]
        q_temp.pop(q_temp.index(-1))
        q_max = np.max(q_temp)
        return q_max
    else:
        q_max = np.max(q_list)
        
get_q_max(test2)

3

### test --------------

## How about making a bigger grid

In [38]:
np.random.choice([0,1,2,3])

1